来自于廖雪峰的[实战教程](http://www.liaoxuefeng.com/wiki/001374738125095c955c1e6d8bb493182103fac9270762a000/0014023080708565bc89d6ab886481fb25a16cdc3b773f0000)，他实现了一个小的web framework。  
其中有一点就是拦截器，对于符合某些url规则的请求，需要做一些操作，比如对于某些关键url，一定要求访问的人具有特定的权限。  


In [2]:
import threading
ctx = threading.local()

以访问URL以'/manage/'开头的请求为例，我们希望这些请求都必须是管理员(`admin`)，那么基本的逻辑代码如下：

In [2]:
def manage_interceptor():
    user = ctx.request.user
    if user and user.admin:
        return True, 'ok'
    else:
        return False, 'You need to login as admin'

最容易想到的就是，在逻辑代码里面内嵌类似如下代码完成：
```
if not ctx.request.path_info.startswith('/manage/'):
    return
```  
即压根不执行函数内部的任何逻辑。

然而有一个问题在于，这段代码具有一定的侵入性，所有的业务代码都要进行这样的判断，而且这种工作几乎都是重复的，同时函数的返回值并不统一。  
那么一个简单的想法就是我给这个函数加一个带参数的装饰器，自动给函数增加关于路径的判断，然后我的框架代码统一对拦截器进行调用。  
我们先来看看装饰器：

In [3]:
import re
_RE_INTERCEPTOR_STARTS_WITH = re.compile(r'^([^\*\?]+)\*?$')
_RE_INTERCEPTOR_ENDS_WITH = re.compile(r'^\*([^\*\?]+)$')

from functools import wraps

def _build_pattern_fn(pattern):
    """
    传入需要匹配的字符串： URL
    返回一个函数，该函数接收一个字符串参数，检测该字符串是否
    符合pattern
    """
    m = _RE_INTERCEPTOR_STARTS_WITH.match(pattern)
    if m:
        return lambda p: p.startswith(m.group(1))
    m = _RE_INTERCEPTOR_ENDS_WITH.match(pattern)
    if m:
        return lambda p: p.endswith(m.group(1))
    raise ValueError('Invalid pattern definition in interceptor.')
    
def interceptor(pattern='/'):
    """
    An @interceptor decorator.
    @interceptor('/admin/')
    def check_admin(req, resp):
        pass
    """

    def _decorator(func):
        @wraps(func)
        def inner(*args, **kw):
            func.__interceptor__ = _build_pattern_fn(pattern)
            return func
        return inner()

    return _decorator

我们只看`def interceptor`，发现它只做了一件事儿：就是给拦截器增加了一个内部方法(Python中的函数本身也是对象哦)用于判断何时需要调用拦截器。

In [4]:
@interceptor('/manage/')
def manage_interceptor():
    user = ctx.request['user']
    if user and user['admin']:
        return True, 'ok'
    else:
        return False, 'You need to login as admin'

好了，拦截器我们已经定义好了，且每个拦截器会被`interceptor`这个装饰器修饰，进而所有的拦截器内部都会有一个方法来判断是不是需要调用它自己，那么我们的统一框架函数就可以这么做了：

In [5]:
interceptor_list = [manage_interceptor,]

ctx.request = {'path_info':'/manage/test', 
               'user':
                   {'name':'fakeuser', 
                    'admin':False}
              } # 虚构一个请求

def run_interceptors():
    for inter_func in interceptor_list:
        if inter_func.__interceptor__(ctx.request['path_info']):
            print '[DEBUG]', 'running', inter_func.__name__
            print inter_func()

run_interceptors()

[DEBUG] running manage_interceptor
(False, 'You need to login as admin')


看上去很好，我们再试一下，新建一个拦截器，针对`/user/`这种URL进行简单的输出操作：

In [6]:
@interceptor('/user/')
def simple_print4user():
    user = ctx.request['user']
    if user:
        return user['name']
    else:
        return 'no user name'

interceptor_list.append(simple_print4user)

In [8]:
ctx.request = {'path_info':'/user/test', 
               'user':
                   {'name':'fakeuser', 
                    'admin':False}
              } # 虚构一个请求

run_interceptors()

[DEBUG] running simple_print4user
fakeuser


到此，我们就完成了一个拦截器模块的功能。我们细致地想一下，所谓的拦截器就是根据URL符合的规则来对函数进行适当的调用。  
所以，我们其实压根不需要使用装饰器(decorator)，简单地做我们只需要一个配置列表：  
```
    [('/user/', simple_print4user), ('/manage/', manage_interceptor)]
```  
我们的框架只需要遍历这个拦截器配置列表就行了，而且我们也不需要辛苦地撰写decorator了，毕竟它的逻辑还是有一点小绕绕的。  
不过再细致地想一下，使用decorator的话有一个明显的好处在于看到代码你就知道它对什么样的URL生效，而不需要去看上面说的配置列表。  

另外，廖雪峰的示例代码中并没有采用我上面写的`run_interceptor`这种简单的循环方式，而是将所有的interceptor顺序两两pack成一个大函数的方式，采用类似`reduce`的思想实现，也很巧妙。感觉他主要是为了展示函数、闭包的用法。

总之，实现一个功能有很多种方法，而且各种方法之间也不一定有优劣之分。